In [ ]:
import requests
import pandas as pd
import re

# attribute_{숫자} 형식을 goodsAttributes로 변경하는 함수
def replace_attribute_key(option_key):
    if re.match(r'attribute_\d+', option_key):
        return 'goodsAttributes'
    return option_key

# Musinsa API로부터 카테고리 데이터 수집
url = "https://display.musinsa.com/display/api/v2/category-menu?gf=A"
res = requests.get(url)
if res.status_code == 200:  # get requests가 제대로 요청 완료된 경우에만 아래의 코드 진행함
    musinsa_data = res.json()['data']['menuItems']  # 상위 및 하위 카테고리 수집을 위함

    # 카테고리 수집
    category_point_url = []  # 카테고리 리스트 [(상위카테고리, 하위카테고리, 하위카테고리명), ...]
    for i in range(3):  # 상위 카테고리 중 [상의, 아우터, 바지]까지만 수집
        menugroups = musinsa_data[i]['menuGroups'][0]['menuItems']
        for j in range(2, len(menugroups)):  # [** 전체, ** 신상] 제외하고, 하위 카테고리 수집
            category_point_url.append((menugroups[j]['landingUrl'].split('/')[-1][:3], menugroups[j]['landingUrl'].split('/')[-1][3:], menugroups[j]['title']))
    for k in range(2):  # 상위 카테고리 중 [원피스/스커트] 수집
        menugroups = musinsa_data[3]['menuGroups'][k]['menuItems']
        for j in range(2, len(menugroups)):  # [** 전체, ** 신상] 제외하고, 하위 카테고리 수집
            category_point_url.append((menugroups[j]['landingUrl'].split('/')[-1][:3], menugroups[j]['landingUrl'].split('/')[-1][3:], menugroups[j]['title']))

    # 카테고리 테이블 생성
    cat_table = pd.DataFrame(category_point_url, columns=['high_cat', 'low_cat', 'low_cat_name'])

# total_dict 생성
total_dict = {}
for i in range(len(category_point_url)):
    high_cat_code, low_cat_code, category_name = category_point_url[i]
    json_data = requests.get(f'https://display.musinsa.com/display/api/v2/categories/ITEM/goods/filter/detail?siteKindId=musinsa&sex=A&originalYn=N&sendLog=true&kidsYn=N&categoryCode={high_cat_code}{low_cat_code}').json()

    category_data = []
    for j in range(len(json_data['data'])):
        key = json_data['data'][j]['key']
        displayText = json_data['data'][j]['displayText']

        # 중복 제거를 위해 set 사용
        filter_contents = set()
        for k in range(len(json_data['data'][j]['filterContents'])):
            code = json_data['data'][j]['filterContents'][k]['code']
            name = json_data['data'][j]['filterContents'][k]['name']
            if name:  # 빈 문자열이 아닌 경우에만 추가
                filter_contents.add((code, name))

        # set을 다시 리스트로 변환하여 저장
        category_data.append({(key, displayText): list(filter_contents)})

    category_data = [item for item in category_data if list(item.values())[0]]  # 빈 리스트이거나 빈 문자열인 경우 제거

    total_dict[category_point_url[i]] = category_data

# total_dict를 데이터프레임으로 변환
rows = []

for (high_cat_code, low_cat_code, category_name), category_data in total_dict.items():
    for item in category_data:
        for (key, displayText), values in item.items():
            for code, name in values:
                row = {
                    'high_cat': high_cat_code,
                    'low_cat': low_cat_code,
                    'low_cat_name': category_name,
                    'option_key': key,
                    'display_name': displayText,
                    'option_code': code,
                    'option_name': name
                }
                rows.append(row)

# 옵션 테이블 생성
option_table = pd.DataFrame(rows)

# cat_table에 고유 인덱스 추가
cat_table = cat_table.reset_index().rename(columns={'index': 'cat_id'})

# option_table과 cat_table을 high_cat, low_cat, low_cat_name으로 매칭
option_table = option_table.merge(cat_table[['cat_id', 'high_cat', 'low_cat', 'low_cat_name']], on=['high_cat', 'low_cat', 'low_cat_name'], how='left')

# option_table에 고유 인덱스 추가
option_table = option_table.reset_index().rename(columns={'index': 'option_id'})

# 필요 없는 열 삭제
option_table = option_table.drop(columns=['high_cat', 'low_cat', 'low_cat_name'])

# apply 함수를 사용하여 option_key 열의 값을 변경
option_table['option_key'] = option_table['option_key'].apply(replace_attribute_key)

# csv 파일로 저장
cat_table.to_csv('category_table.csv', sep='\t', index=False)
option_table.to_csv('option_table.csv', sep='\t', index=False)

# cat_table 및 option_table 결과 출력
print(cat_table)
print(option_table)

In [3]:
import pandas as pd
import requests

# csv 파일 가져오기
cat_table = pd.read_csv('category_table.csv', sep='\t')
option_table = pd.read_csv('option_table.csv', sep='\t')
cat_table, option_table

(    high_cat  low_cat low_cat_name
 0          1        5    맨투맨/스웨트셔츠
 1          1        2      셔츠/블라우스
 2          1        4       후드 티셔츠
 3          1        6       니트/스웨터
 4          1        3    피케/카라 티셔츠
 5          1       10      긴소매 티셔츠
 6          1        1      반소매 티셔츠
 7          1       11      민소매 티셔츠
 8          1       13       스포츠 상의
 9          1        8        기타 상의
 10         2       22        후드 집업
 11         2        1     블루종/MA-1
 12         2        2   레더/라이더스 재킷
 13         2       25        무스탕/퍼
 14         2       17       트러커 재킷
 15         2        3   슈트/블레이저 재킷
 16         2       20          카디건
 17         2       19       아노락 재킷
 18         2       23      플리스/뽀글이
 19         2       18      트레이닝 재킷
 20         2        4      스타디움 재킷
 21         2        8       환절기 코트
 22         2        7     겨울 싱글 코트
 23         2       24     겨울 더블 코트
 24         2        9     겨울 기타 코트
 25         2       13  롱패딩/롱헤비 아우터
 26         2       12  숏패딩/

In [7]:
option_table['option_key'].unique()

array(['goodsAttributes', 'discountRate', 'price', 'color',
       'standardSize', 'measureSize'], dtype=object)

In [10]:
# 'option_key'가 'price'인 행 찾기 ---> 크롤링단계에서 필요하지 않다고 생각되어 제거함
option_table[option_table['option_key'] == 'price']

,option_id,option_key,display_name,option_code,option_name,cat_id
50,50,price,가격,100000^200000,10만원 ~ 20만원,0
51,51,price,가격,0^50000,5만원 이하,0
52,52,price,가격,300000^0,30만원 이상,0
53,53,price,가격,50000^100000,5만원 ~ 10만원,0
54,54,price,가격,200000^300000,20만원 ~ 30만원,0
...,...,...,...,...,...,...
4602,4602,price,가격,100000^200000,10만원 ~ 20만원,46
4603,4603,price,가격,0^50000,5만원 이하,46
4604,4604,price,가격,300000^0,30만원 이상,46
4605,4605,price,가격,50000^100000,5만원 ~ 10만원,46


In [12]:
# 'display_name'이 '실측 사이즈'인 행 제거
option_table_filtered = option_table[option_table['display_name'] != '실측 사이즈']
# 'option_key'가 'price'인 행 제거
option_table_filtered = option_table_filtered[option_table_filtered['option_key'] != 'price']

# 'option_key'가 'discountRate'인 경우 'discountRateCode'로 변경 (color, standardSize도 마찬가지로 적용)
option_table_filtered['option_key'] = option_table_filtered['option_key'].replace('discountRate', 'discountRateCode')
option_table_filtered['option_key'] = option_table_filtered['option_key'].replace('color', 'colorCodes')
option_table_filtered['option_key'] = option_table_filtered['option_key'].replace('standardSize', 'standardSizeOptions')
option_table = option_table_filtered
option_table

,option_id,option_key,display_name,option_code,option_name,cat_id
0,0,goodsAttributes,소매기장,11^228,긴소매,0
1,1,goodsAttributes,소매기장,11^227,3/4길이,0
2,2,goodsAttributes,소매기장,11^225,반소매,0
3,3,goodsAttributes,소매기장,11^224,민소매,0
4,4,goodsAttributes,핏,2^87,슬림,0
...,...,...,...,...,...,...
4649,4649,standardSizeOptions,표준 사이즈,XXL,2XL 이상,46
4650,4650,standardSizeOptions,표준 사이즈,S,S,46
4651,4651,standardSizeOptions,표준 사이즈,L,L,46
4652,4652,standardSizeOptions,표준 사이즈,XS,XS,46


In [ ]:
# 수집한 데이터를 저장할 리스트 초기화
final_rows = []

# 최종 데이터 사용 예시
for _, row in option_table.iterrows():
    option_key = row['option_key']
    goods_attributes = row['option_code']
    cat_id = row['cat_id']
    high_cat = cat_table.loc[cat_id, 'high_cat']
    low_cat = cat_table.loc[cat_id, 'low_cat']
    categorycode = f'{high_cat}{low_cat}'
    for page in range(1, 2):
        print(f'{row["display_name"]}_{row["option_name"]} : {page} 페이지 크롤링 중입니다.')
        data_url = f'https://display.musinsa.com/display/api/v2/categories/ITEM/goods?siteKindId=musinsa&sex=A&sortCode=pop_category&{option_key}={goods_attributes}&categoryCode={categorycode}&size=60&page={page}'
        response = requests.get(data_url)
        if response.status_code == 200:
            data = response.json()
            prod_total = data['data']['goodsList']
            for i in range(len(prod_total)):
                final_row = {
                    'category_id': cat_id,
                    'option_id': row['option_id'],
                    'product_name': prod_total[i]['goodsName'],
                    'price': prod_total[i]['price'],
                    'product_image': prod_total[i]['imageUrl'],
                    'product_url': prod_total[i]['linkUrl'],
                    'product_type':1,
                    'product_state':1
                }
                final_rows.append(final_row)

# 최종 데이터프레임 생성
final_df = pd.DataFrame(final_rows)

# 최종 데이터프레임 출력
final_df